In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import plotly.graph_objects as go

url = 'D:\Intern\Jupyter Notebook\Indonesia_Stocks\data\BBCA_stock_data.csv'
data = pd.read_csv(url)

data

<>:10: SyntaxWarning:

invalid escape sequence '\I'

<>:10: SyntaxWarning:

invalid escape sequence '\I'

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23304\839295225.py:10: SyntaxWarning:

invalid escape sequence '\I'



,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-01,5200.0,5200.0,5200.0,5200.0,4629.853516,0
1,2019-01-02,5200.0,5245.0,5200.0,5240.0,4665.468750,35956000
2,2019-01-03,5200.0,5220.0,5115.0,5180.0,4612.047363,72358000
3,2019-01-04,5175.0,5205.0,5125.0,5205.0,4634.306152,51465000
4,2019-01-07,5265.0,5325.0,5245.0,5245.0,4669.918945,73438000
...,...,...,...,...,...,...,...
1412,2024-10-08,10200.0,10500.0,10150.0,10400.0,10400.000000,99622900
1413,2024-10-09,10400.0,10525.0,10400.0,10425.0,10425.000000,56835200
1414,2024-10-10,10425.0,10500.0,10400.0,10500.0,10500.000000,51080000
1415,2024-10-11,10575.0,10575.0,10375.0,10375.0,10375.000000,44632100


In [16]:
data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-01,5200.0,5200.0,5200.0,5200.0,4629.853516,0
1,2019-01-02,5200.0,5245.0,5200.0,5240.0,4665.468750,35956000
2,2019-01-03,5200.0,5220.0,5115.0,5180.0,4612.047363,72358000
3,2019-01-04,5175.0,5205.0,5125.0,5205.0,4634.306152,51465000
4,2019-01-07,5265.0,5325.0,5245.0,5245.0,4669.918945,73438000
5,2019-01-08,5230.0,5320.0,5205.0,5240.0,4665.468750,66247500
6,2019-01-09,5300.0,5320.0,5225.0,5255.0,4678.823730,84871000
7,2019-01-10,5300.0,5300.0,5190.0,5255.0,4678.823730,114314500
8,2019-01-11,5280.0,5300.0,5235.0,5250.0,4674.372070,84826500
9,2019-01-14,5220.0,5235.0,5150.0,5200.0,4629.853516,87547500


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1417 entries, 0 to 1416
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1417 non-null   object 
 1   Open       1417 non-null   float64
 2   High       1417 non-null   float64
 3   Low        1417 non-null   float64
 4   Close      1417 non-null   float64
 5   Adj Close  1417 non-null   float64
 6   Volume     1417 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 77.6+ KB


In [18]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [19]:
data.Date = pd.to_datetime(data.Date)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1417 entries, 0 to 1416
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1417 non-null   datetime64[ns]
 1   Open       1417 non-null   float64       
 2   High       1417 non-null   float64       
 3   Low        1417 non-null   float64       
 4   Close      1417 non-null   float64       
 5   Adj Close  1417 non-null   float64       
 6   Volume     1417 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 77.6 KB


In [20]:
# Sort data by date to ensure proper time series ordering
data = data.sort_values('Date')

# Feature selection: using 'Open', 'High', 'Low', 'Volume', and 'Adj Close' (with lag)
# Create lag features for 'Adj Close' to include previous days' values
data['Adj Close_Lag1'] = data['Adj Close'].shift(1)
data['Adj Close_Lag2'] = data['Adj Close'].shift(2)

# Drop rows with NaN values created by lagging
data_clean = data.dropna()

# Features and target
features = ['Open', 'High', 'Low', 'Volume', 'Adj Close_Lag1', 'Adj Close_Lag2']
target = 'Adj Close'

# Split data into training and testing sets (80% training, 20% testing based on date)
train_size = int(len(data_clean) * 0.8)
train_data = data_clean.iloc[:train_size]
test_data = data_clean.iloc[train_size:]

# Separate features and target for both train and test sets
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

X_train.head(), y_train.head()


(     Open    High     Low    Volume  Adj Close_Lag1  Adj Close_Lag2
 2  5200.0  5220.0  5115.0  72358000     4665.468750     4629.853516
 3  5175.0  5205.0  5125.0  51465000     4612.047363     4665.468750
 4  5265.0  5325.0  5245.0  73438000     4634.306152     4612.047363
 5  5230.0  5320.0  5205.0  66247500     4669.918945     4634.306152
 6  5300.0  5320.0  5225.0  84871000     4665.468750     4669.918945,
 2    4612.047363
 3    4634.306152
 4    4669.918945
 5    4665.468750
 6    4678.823730
 Name: Adj Close, dtype: float64)

In [21]:
# Initialize Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

mae, rmse

(np.float64(83.08734930879747), np.float64(107.05232785747417))

In [22]:
# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                 open=data['Open'],
                 high=data['High'],
                 low=data['Low'],
                 close=data['Close'],
                 name='BBCA')])

# Update layout
fig.update_layout(
    title='Candlestick Chart for BBCA Stock',
    xaxis_title='Date',
    yaxis_title='Stock Price (IDR)',
    xaxis_rangeslider_visible=False
)

fig.show()
